In [4]:

import os
import pandas as pd
import numpy as np
import warnings
import scipy as sp 
pd.options.display.max_columns
anime_type = 'TV'
anime_name = 'Naruto'
num_recommendations = 4
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

from sklearn.metrics.pairwise import cosine_similarity
anime_data = pd.read_csv('C:\\Users\\udit hasija\\Documents\\Project Anime\\data\\anime.csv')
rating_data = pd.read_csv('C:\\Users\\udit hasija\\Documents\\Project Anime\\data\\rating.csv')
anime_data = anime_data[~np.isnan(anime_data['rating'])]
anime_data['genre'] = anime_data['genre'].fillna(anime_data['genre'].dropna().mode().values[0])
anime_data['type'] = anime_data['type'].fillna(anime_data['type'].dropna().mode().values[0])
rating_data['rating'] = rating_data['rating'].apply(lambda x: np.nan if x==-1 else x)
atype = anime_type
anime_data = anime_data[anime_data['type']==atype]
anime_full = pd.merge(anime_data , rating_data , on = 'anime_id')
anime_full = anime_full.rename(columns= {'rating_x': 'rating' , 'rating_y':'user_rating'})
anime_full = anime_full[['user_id', 'name', 'user_rating']]
anime_7500= anime_full[anime_full.user_id <= 25000]
pivot = anime_7500.pivot_table(index=['user_id'], columns=['name'], values='user_rating')
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
pivot_n.fillna(0 , inplace= True)
pivot_n = pivot_n.T
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]
pivsparse = sp.sparse.csr_matrix(pivot_n.values)
similarity = cosine_similarity(pivsparse)
ani_df = pd.DataFrame(similarity, index = pivot_n.index, columns = pivot_n.index)
recommendations = {'name': [], 'match' : [] ,'synopsis': [] ,'episodes':[]}
number = 1
for anime in ani_df.sort_values(by = anime_name, ascending = False).index[1:int(num_recommendations)+1]:
    synop = anime_data[anime_data['name'] == anime]
    synopsis = synop['Synopsis'].values[0]
    perc = round(ani_df[anime][anime_name]*100,2)
    epi = synop['episodes'].values[0]
    recommendations['name'].append(anime)
    recommendations['synopsis'].append(synopsis)
    recommendations['episodes'].append(int(epi))
    recommendations['match'].append(perc)
    number +=1  


In [5]:
print(recommendations)

{'name': ['Naruto: Shippuuden', 'Bleach', 'InuYasha', 'Dragon Ball Z'], 'match': [28.34, 19.98, 10.69, 10.25], 'synopsis': ['It has been two and a half years since Naruto Uzumaki left Konohagakure, the Hidden Leaf Village, for intense training following events which fueled his desire to be stronger. Now Akatsuki, the mysterious organization of elite rogue ninja, is closing in on their grand plan which may threaten the safety of the entire shinobi world.\n \nAlthough Naruto is older and sinister events loom on the horizon, he has changed little in personality—still rambunctious and childish—though he is now far more confident and possesses an even greater determination to protect his friends and home. Come whatever may, Naruto will carry on with the fight for what is important to him, even at the expense of his own body, in the continuation of the saga about the boy who wishes to become Hokage.', "Ichigo Kurosaki is an ordinary high schooler—until his family is attacked by a Hollow, a c